# IBM Code Model Asset Exchange Review Text Generator

- https://github.com/IBM/MAX-Review-Text-Generator


## Setup

1. In a terminal window, run the following commands to clone the repository and also download the model artifact for the Review Text Generator:
    ```
    git clone https://github.com/IBM/MAX-Review-Text-Generator.git
    
    curl -O http://max-assets.s3-api.us-geo.objectstorage.softlayer.net/keras/generative_lang_model/generative_lang_model.h5
    ```    

2. Run the notebook `jupyter notebook .`

In [ ]:
# This notebook has been tested with Python version 3.6
!python --version

In [ ]:
# This notebook has been tested with tensorflow 1.10.1, tensorflowjs 0.6.0, and numpy 1.14.5
!pip show tensorflow tensorflowjs numpy

In [ ]:
# Uncomment to install the packages needed

# !pip install -Iv tensorflow
# !pip install -Iv tensorflowjs
# !pip install -Iv numpy

# Restart the kernel after installation completes.

<br>

Update the variable with the appropriate directory path to the model

In [ ]:
# full path to keras model
keras_model_file = '/Users/va/models/review-text-generator/generative_lang_model.h5'


<br>

Import libraries used in this notebook

In [ ]:
import numpy as np
import json
import pathlib
import os
import time
from timeit import default_timer as timer
import tensorflow as tf

In [ ]:
print('TF versions:', tf.GIT_VERSION, tf.VERSION)


<br>

## Load Keras model

In [ ]:
# load the keras model
def load_model(path):
    model = tf.keras.models.load_model(
        path,
        custom_objects=None,
        compile=True
    )
    return model


In [ ]:
model = load_model(keras_model_file)


<br>

## Run inference

<br>

Set the helper functions

In [ ]:
# length required for the seed text. 
# seed text will be padded or truncated as needed
SEED_TEXT_LEN = 256


# https://github.com/IBM/MAX-Review-Text-Generator/blob/master/core/backend.py#L28
def load_assets(path):
    p1 = '{}/char_indices.txt'.format(path)
    print(p1)
    with open(p1) as f:
        char_indices = json.loads(f.read())
        chars = sorted(char_indices.keys())
        num_chars = len(chars)
        
    p2 = '{}/indices_char.txt'.format(path)
    print(p2)
    with open(p2) as f:
        indices_char = json.loads(f.read())

    return char_indices, chars, num_chars, indices_char


# https://github.com/IBM/MAX-Review-Text-Generator/blob/master/core/backend.py#L36
def _sample(preds, temperature=.6):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


# https://github.com/IBM/MAX-Review-Text-Generator/blob/master/core/backend.py#L45
def predict(sentence, gen_chars=50):
    # Generate text based on seed text.
    sentence = sentence.lower()
    for t, char in enumerate(sentence):
        if char not in char_indices:
            print("Bad char {} at position {}".format(char, t))
            raise ValueError(
                    "Unexpected character '{}' at position {}. "
                    "Only lowercase ASCII characters, spaces, "
                    "and basic punctuation are supported.".format(char, t))

    # The text passed into the model must be exactly SEED_TEXT_LEN
    # characters long, or the model will crash. Pad or truncate.
    if len(sentence) > SEED_TEXT_LEN:
        sentence = sentence[:SEED_TEXT_LEN]
    else:
        sentence = sentence.rjust(SEED_TEXT_LEN)

    generated = ''
    start = timer()
    
    for i in range(gen_chars):
        x = np.zeros((1, SEED_TEXT_LEN, num_chars))

        for t, char in enumerate(sentence):
            x[0, t, char_indices[char]] = 1.

        preds = model.predict(x, verbose=0)[0]

        next_index = _sample(preds)
        next_char = indices_char[str(next_index)]

        generated += next_char
        sentence = sentence[1:] + next_char
    
    end = timer()
    print('predict: {}'.format(end - start))

    return generated

<br>

Set the path of the model assets and load the assets.

In [ ]:
# path to the assets directory in the github repository
model_assets = '/Users/va/git/MAX-Review-Text-Generator/assets'

char_indices, chars, num_chars, indices_char = load_assets(model_assets)


<br>

Run prediction

In [ ]:
# sentence = 'Came here last Friday with my friends. Got there around 8:30 and got seated right away. Parking'
sentence = 'heart be still i loved this place. way better than i expected. i had the spicy noodles and they were delicious, flavor great and quality was on point. for desert the sticky rice with mango, i dream about it now. highly recommend if you are in the mood for '

generated = predict(sentence)

print (sentence)
print (generated)

<br>
<hr>

## Converting to a web-friendly format

[https://github.com/tensorflow/tfjs-converter](https://github.com/tensorflow/tfjs-converter)

#### keras model

```
tensorflowjs_converter \
    --input_format=keras \
    generative_lang_model.h5 \
    converterout
```

set the `web_asset_path` accordingly

In [ ]:
# full path to directory where converter output will be saved
web_asset_dir = '/Users/va/models/review-text-generator/web_assets'

# create directory if it does not exist
pathlib.Path(web_asset_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
# set appropriate keras model path and desired output path for web format

!tensorflowjs_converter \
    --input_format=keras \
    {keras_model_file} \
    {web_asset_dir}


In [ ]:
print("Web asset directory {}:".format(web_asset_dir))

web_assets = os.listdir(web_asset_dir)
web_assets.sort()

for file in web_assets:
    file_stat = os.stat("{}/{}".format(web_asset_dir,file))
    print(" {} {} {:>20}".format(file.ljust(30), time.ctime(file_stat.st_mtime), file_stat.st_size))
